### 행정구역 정보 분석 및 시각화
1. pandas의 read_csv() 함수로 csv file을 읽어서 DataFrame 객체로 생성하고 데이터 확인하기
2. Data Selection : loc[] / iloc[] 를 사용해서 특정 Row와 Column 선택하기
3. 컬럼명 변경하고, 컬럼 삭제하기
4. DataFrame 객체를 Excel file로 저장하기
5. Data Grouping : groupby() 함수를 사용해서 그룹핑하기
6. 상관관계 분석 : corr() 함수를 사용해서 인구수와 면적간의 상관관계 
7. 시각화 : seaborn의 barplot() 함수를 사용해서 Plot 그리기

In [ ]:
import pandas as pd
print(pd.__version__)

In [ ]:
data = pd.read_csv('data/data_draw_korea.csv')
print(type(data))
# 행과열 확인
print(data.shape)

In [ ]:
data.head()

In [ ]:
data.tail(3)

In [ ]:
data.sample(3)

In [ ]:
print(f'컬럼명 : {data.columns}')
print(f'인덱스 : {data.index}')

In [ ]:
print(type(data.values))
data.values

In [ ]:
data.info()

In [ ]:
# 모든 컬럼의 타입 확인
data.dtypes

In [ ]:
# 특정컬럼(인구수)의 타입 확인
print(type(data['인구수']))
data['인구수'].dtypes

In [ ]:
# import numpy as np
# data.describe(include=[np.number]) 와 동일한 결과
data.describe()  

In [ ]:
data.describe(include=[object])

In [ ]:
# 해당 조건식과 매칭되는 Row data 출력
data.loc[data['행정구역'] == '동구']

In [ ]:
data.describe(include='all')

### Data Selection

In [ ]:
# 인구수 1개의 컬럼 선택
print(type(data['인구수']))
data['인구수']

In [ ]:
# 인구수 컬럼의 집계함수
print(f"인구수 최대값 {data['인구수'].max():,}")
print(f"인구수 최소값 {data['인구수'].min():,}")
print(f"인구수 평균 {round(data['인구수'].mean()):,}")
print(f"인구수 표준편차 {round(data['인구수'].std()):,}")
print(f"인구수 중간값 {data['인구수'].median():,}")
data['인구수'].quantile([0.25, 0.75])

In [ ]:
# 인구수가 가장 많은 행정구역?
max_prop = data['인구수'].max()
#data['인구수'] == max_prop
data.loc[data['인구수'] == max_prop]

In [ ]:
# 인구수가 가장 적은 행정구역?
min_prop = data['인구수'].min()
data.loc[data['인구수'] == min_prop]

In [ ]:
# unique한 광역시도명
print(len(data['광역시도'].unique()))
data['광역시도'].unique()

In [ ]:
# 광역시도별 Row Counting
data['광역시도'].value_counts()

In [ ]:
#경기도에 속한 행정구역명, 인구수, 면적 선택하기
#data.loc[row , col]
gy_df=data.loc[data['광역시도'] == '경기도',['행정구역','인구수','면적']].sort_values(by='인구수',ascending=False).reset_index(drop=True)

#새로운 컬럼을 생성
gy_df['인구수2']=gy_df['인구수'].map('{:,}'.format)
gy_df['면적2']=gy_df['면적'].map('{:.2f}'.format)
gy_df

In [ ]:
data['면적'].max()

In [ ]:
data.loc[data['면적'] == data['면적'].max()]

In [ ]:
# 조건과 매핑되는 새로운 DataFrame 생성
area_1000 = data.loc[data['면적'] > 1000].copy()
#df.loc[df['A'] > 2, 'B'] = new_val
area_1000.loc[:,'면적2']=area_1000['면적'].map('{:,.2f}'.format)
area_1000.sort_values(by="면적",ascending=False)

In [ ]:
#경기도 인구수의 평균과 표준편차
print(f"경기도 인구수 평균 = {gy_df['인구수'].mean()}")
print(f"경기도 인구수 편차 = {gy_df['인구수'].std()}")

In [ ]:
#광역시도명을 인자로 받아서 해당 광역시에 속한 행정구역들의 인구수의 평균과 편차 구하기
def pop_mean_std(sido_name):
    sido_df = data.loc[data['광역시도'] == sido_name,'인구수':'행정구역']
    print(f"{sido_name} 인구수 평균 = {sido_df['인구수'].mean()}")
    print(f"{sido_name} 인구수 편차 = {sido_df['인구수'].std()}")

In [ ]:
pop_mean_std('서울특별시')

In [ ]:
pop_mean_std('강원도')

In [ ]:
for sido_name in data['광역시도'].unique():
    pop_mean_std(sido_name)

### Data Selection 
* loc[]
* iloc[]

In [ ]:
#loc[] 사용
#인덱스가 20 부터 25까지
data.loc[20:25]

In [ ]:
#iloc[] 사용
#인덱스가 20 부터 25까지
data.iloc[20:25]

In [ ]:
#iloc[] 사용
#인덱스가 20 부터 25까지 컬럼이 인구수부터 행정구역 까지
data.iloc[20:25,1:8]

In [ ]:
#loc[] 사용
#index가 40,55,60 이고, column이 인구수,광역시도,행정구역
data.loc[[40,55,60],['인구수','광역시도','행정구역']]

In [ ]:
#인덱스가 0인 행 하나만 선택
print(type(data.loc[0]))
data.loc[0]

In [ ]:
#열 하나만 선택
print(type(data['면적']))
data['면적'].head()

### 컬럼명 변경하기, 컬럼을 인덱스 전환하기
* rename() 함수
  - inPlace=False (default) 원본 DataFrame객체를 변경하지 않고, 변경한 결과만 출력한다.
  - inPlace=True 원본 DataFrame객체를 변경하고, 변경한 결과는 출력하지 않는다.

In [ ]:
data.columns

In [ ]:
# 'Unnamed: 0' 컬럼명을 'seq' 로 변경하기
data.rename(columns={'Unnamed: 0':'seq'}, inplace=True)

In [ ]:
data.head(2)

In [ ]:
# seq 컬럼을 인덱스로 변경한다
data.set_index('seq', inplace=True)

In [ ]:
data.head(2)

### DataFrame을 Excel file로 저장하기
* 인구수가 평균보다 작은 행정구역, 광역시도, 인구수를 선택해서 DataFrame생성하기
* DataFrame의 to_excel() 함수 사용

In [ ]:
pop_mean_value=data['인구수'].mean()
print(pop_mean_value)
print(round(pop_mean_value))
print(f'{pop_mean_value:.2f}')

pop_mean_lt_df = data.loc[data['인구수'] < pop_mean_value,['광역시도','행정구역','인구수']]\
.sort_values(by=['광역시도','인구수'],ascending=[True,False])\
.reset_index(drop=True)
pop_mean_lt_df.head(3)
#excel 파일로 저장
pop_mean_lt_df.to_excel('data/평균인구수미만지역.xlsx')

### GroupBy 기능 사용하기
* 광역시도별 행정구역의 인구수 합계
* ~별에 해당하는 컬럼명이나 컬럼값을 groupby() 함수의 인자로 전달한다. 
* Series 객체의 groupby(data['광역시도']) 함수는 컬럼의 값을 인자로 전달하고 
* DataFrame 객체의 groupby('광역시도') 함수는 컬럼명을 인자로 전달합니다. 

In [ ]:
# Series 객체 사용 - 광역시도별 인구수의 합계 
# SeriesGroupBy object
data['인구수'].groupby(data['광역시도']).sum().sort_values(ascending=False)

In [ ]:
# DataFrame 객체 사용 - 광역시도별 인구수의 합계 
# DataFrameGroupBy object
data.groupby('광역시도').sum(numeric_only=True)

In [ ]:
data.groupby('광역시도')['인구수'].sum().sort_values(ascending=False)

In [ ]:
# 광역시도별 행정구역별 인구수의 합계
group_df = data.groupby(['광역시도','행정구역'])['인구수'].sum()
group_df

In [ ]:
group_df.to_excel('data/광역시도별행정구역별인구수의합계.xlsx', sheet_name='인구수')

#### 엑셀 수치데이터에 콤마찍기
* [ExcelWriter](https://pandas.pydata.org/docs/reference/api/pandas.ExcelWriter.html)

In [ ]:
!pip show xlsxwriter

In [ ]:
# Create a Pandas Excel writer using XlsxWriter as the engine.
writer = pd.ExcelWriter('data/광역시도별인구수의합계1.xlsx', engine='xlsxwriter')

# Convert the dataframe to an XlsxWriter Excel object.
group_df.to_excel(writer, sheet_name='인구수합계')

# Get the xlsxwriter workbook and worksheet objects.
workbook  = writer.book
worksheet = writer.sheets['인구수합계']

# Set a currency number format for a column.
num_format = workbook.add_format({'num_format': '#,###'})
worksheet.set_column('C:C', None, num_format)

# Close the Pandas Excel writer and output the Excel file.
writer.close() #_save()

#### 상관계수 (Correlation Coefficient)
* 인구수와 면적 데이터간에 관련성이 있는지 살펴보기 위해서 상관계수 구하기
* 상관계수 값은 -1 ~ 1 사이의 값이며, 0에 가까울 수록 관련성이 낮고, 1에 가까울 수록 관련성이 높다. 
* 음수는 반비례 (면적이 넓은 반면 인구수는 적은 경우), 양수는 비례 (면적이 넓고, 인구수도 높은 경우)
* corr() 함수

In [ ]:
#전국의 면적과 인구수의 상관계수 구하기
data['면적'].corr(data['인구수'])

In [ ]:
#서울의 면적과 인구수의 상관계수 구하기
seoul_df = data.loc[data['광역시도'] == '서울특별시']
seoul_df['면적'].corr(seoul_df['인구수'])

In [ ]:
#경기도의 면적과 인구수의 상관계수 구하기
#gy_df = data.loc[data['광역시도'] == '경기도']
gy_df['면적'].corr(gy_df['인구수'])

#### Groupby 한 DataFrame에 집계함수를 여러개 설정하기
* agg() 함수 
* agg(['max','mean','std'])

In [ ]:
data.groupby('광역시도')['인구수'].agg(['max','mean','std','min']).fillna(0)

In [ ]:
# agg() 함수 사용 - 그룹핑한 데이터를 여러개의 집계함수를 사용해서 새로운 DataFrame 객체 생성하기
group_agg_df = data.groupby('광역시도')['인구수'].agg(['max','mean','std'])\
.fillna(0).sort_values(by='std',ascending=False)
group_agg_df

In [ ]:
group_agg_df.style.format('{0:.2f}')

In [ ]:
# 컬럼의 값을 변경 map() 함수를 사용하여 포맷하기
group_agg_df['mean'] = group_agg_df['mean'].map('{:.2f}'.format)
group_agg_df['std'] = group_agg_df['std'].map('{:.2f}'.format)
group_agg_df

#### 시각화
* %matplotlib inline 설정 (jupyter 에서는 show() 함수를 호출하지 않아도 plot이 출력된다)
* 한글폰트 설정이 필요함
* Plot에 대한 설정은 matplotlib의 함수를 사용하고, Plot을 그려주는 기능은 seaborn()의 함수를 사용합니다. 

In [ ]:
%matplotlib inline

In [ ]:
import matplotlib
import matplotlib.pyplot as plt
import matplotlib.font_manager as fm
import seaborn as sns

print('matplotlib ', matplotlib.__version__)
print('seaborn ', sns.__version__)

In [ ]:
# for font in fm.fontManager.ttflist:
#     print((font.name, font.fname))
# 폰트이름과 폰트파일 정보 출력 list comprehension 
[ (font.name, font.fname) for font in fm.fontManager.ttflist if 'Mal' in font.name ]

In [ ]:
# font name을 알고 있다면 생략가능
#한글폰트 path 설정
font_path = 'C:\\windows\\Fonts\\malgun.ttf'
#font의 파일정보로 font name 을 알아내기
font_prop = fm.FontProperties(fname=font_path).get_name()
print(font_prop)

In [ ]:
# matplotlib의 rc(run command) 함수를 사용해서 font name 설정
matplotlib.rc('font', family=font_prop)

#### Figure, Axes, Plot 
* Axes는 Plot이 실제 그려지는 공간
* Figure는 Axes 보다 더 상위의 공간
  : Figure을 여러개의 Axes로 분할해서 Plot를 그릴 수 있다
* 비교) Axis는 X 축, Y축  
* seaborn의 barplot() 함수(막대그래프) 사용 

In [ ]:
seoul_df.head(3)

In [ ]:
figure,(axes1, axes2) = plt.subplots(nrows=2, ncols=1)
figure.set_size_inches(18,12)
print(figure)
print(axes1)
print(axes2)

sns.barplot(x='행정구역',y='인구수',data=seoul_df.sort_values(by='인구수',ascending=False),ax=axes1, hue='행정구역')
#sns.barplot(x='행정구역',y='인구수',data=seoul_df,ax=axes1, hue='행정구역', hue_order="인구수")
sns.barplot(x='행정구역',y='면적',data=seoul_df.sort_values(by='면적',ascending=False), ax=axes2, hue='행정구역')

In [ ]:
# 광역시도 이름을 인자로 받아서 인구수와 면적을 그려주는 함수
def show_pop_area(sido_name):
    sido_df = data.loc[data['광역시도'] == sido_name]
    figure, (axes1,axes2) = plt.subplots(nrows=2, ncols=1)
    figure.set_size_inches(18,12)
    
    pop_plot = sns.barplot(x='행정구역', y='인구수', data=sido_df.sort_values(by='인구수',ascending=False), ax=axes1,hue='행정구역')
    pop_plot.set_title(f'{sido_name} 행정구역별 인구수')
    area_plot = sns.barplot(x='행정구역', y='면적', data=sido_df.sort_values(by='면적',ascending=False), ax=axes2,hue='행정구역')
    area_plot.set_title(f'{sido_name} 행정구역별 면적')

In [ ]:
show_pop_area('경기도')

In [ ]:
show_pop_area('부산광역시')

In [ ]:
# 전국데이터의 광역시도의 인구수 
figure, axes1 = plt.subplots(1,1)
figure.set_size_inches(18,12)
sns.barplot(x='광역시도', y='인구수', data=data, ax=axes1, hue='광역시도')

In [ ]:
# image 파일로 저장하기
# 바깥쪽 여백 제거
figure.savefig('data/전국인구수.png', bbox_inches='tight')
figure.savefig('data/전국인구수2.png')
#plt.savefig('data/전국인구수3.png', dpi=300, bbox_inches='tight')

In [ ]:
print(type(seoul_df["인구수"].items()))
seoul_df["인구수"].items()

In [ ]:
for v in seoul_df["인구수"].items():
    print(v, v[1])

In [ ]:
figure, ax1 = plt.subplots(nrows=1, ncols=1)
figure.set_size_inches(18,12)

sns.barplot(data=seoul_df, x="행정구역", y="인구수", ax=ax1, hue='행정구역')
#y축의 label값에 ,(콤마) 출력하기
ax1.get_yaxis().set_major_formatter(plt.FuncFormatter(lambda x, loc: "{:,}".format(int(x))))

for item in ax1.get_xticklabels(): 
    item.set_rotation(90)
for i, v in enumerate(seoul_df["인구수"].items()):
    ax1.text(i ,v[1], "{:,}".format(v[1]), color='m', va ='bottom', rotation=45)
plt.tight_layout()

In [ ]:
!pip install streamlit

In [ ]:
import matplotlib.pyplot as plt

plt.figure(figsize=(10, 8))

plt.scatter(data['x'], data['y'], s=data['인구수']/1000, alpha=0.6)
for i, name in enumerate(data['행정구역']):
    plt.text(data['x'][i], data['y'][i], name, fontsize=9)

plt.title('지역별 인구수 분포 (상대좌표 기반)')
plt.xlabel('x')
plt.ylabel('y')
plt.grid(True)
plt.show()


In [ ]:
import seaborn as sns
import pandas as pd
import numpy as np

# 피벗 테이블로 좌표 기준 데이터 변환
pivot = data.pivot(index='y', columns='x', values='인구수')

plt.figure(figsize=(10, 8))
sns.heatmap(pivot, annot=True, fmt=".0f", cmap='Reds')
plt.title('지역별 인구수 Heatmap')
plt.show()

In [ ]:
import plotly.express as px

fig = px.scatter(data, x='x', y='y', size='인구수', text='행정구역', title='인구수에 따른 지역 분포')
fig.update_traces(textposition='top center')
fig.show()


In [ ]:
import matplotlib.pyplot as plt

plt.figure(figsize=(12, 8))

# 버블 크기는 인구수 기준
plt.scatter(data['x'], data['y'], s=data['인구수']/500, alpha=0.6, color='cornflowerblue', edgecolors='black')

# 지역명 표시
for idx, row in data.iterrows():
    plt.text(row['x'], row['y'], row['shortName'], fontsize=9, ha='center', va='center')

plt.gca().invert_yaxis()  # y축 뒤집기 (지리적 느낌 살림)
plt.xticks([])            # x축 제거
plt.yticks([])            # y축 제거
plt.grid(True, linestyle='--', alpha=0.5)
plt.title('지역별 인구 분포 (Grid Map)', fontsize=15)
plt.show()


In [ ]:
seoul_df = data.loc[data['광역시도'] == '서울특별시']
# 버블 크기는 인구수 기준
plt.scatter(seoul_df['x'], seoul_df['y'], s=seoul_df['인구수']/500, alpha=0.6, color='cornflowerblue', edgecolors='black')

# 지역명 표시
for idx, row in seoul_df.iterrows():
    plt.text(row['x'], row['y'], row['shortName'], fontsize=9, ha='center', va='center')

plt.gca().invert_yaxis()  # y축 뒤집기 (지리적 느낌 살림)
#plt.xticks([])            # x축 제거
#plt.yticks([])            # y축 제거
plt.grid(True, linestyle='--', alpha=0.5)
plt.title('지역별 인구 분포 (Grid Map)', fontsize=15)
plt.show()